# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


from api_keys import g_key
from pprint import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path=os.path.join('..','WeatherPy','city_data.csv')
df=pd.read_csv(path)
df1=df.drop(['Unnamed: 0'], axis=1)



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations=df1[['Lat','Lon']]
rating=df1['Humidity']
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=max(rating),point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
df2=df1.loc[(df1['Max Temp (f)']>70)&(df1['Max Temp (f)']<80)]
df3=df2.loc[df1['Wind Speed (mph)']<10]
df4=df3.loc[df1['Cloudiness']==0]
df5=df4.dropna(how='any')

locations=df5[['Lat','Lon']]
rating=df5['Humidity']
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=max(rating),point_radius=3)
fig.add_layer(heat_layer)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=df5
hotel_df['Hotel Name']=""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lon=row['Lon']
    location=lat,lon
    city=row['City']
    
    params = {"location": f"{lat},{lon}",
    "type": "lodging",
    "radius": 50000,
    "key": g_key}
   
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")

    response = requests.get(base_url, params).json()
#     pprint(response)
    
#     extract results
    results = response['results']
#     pprint(results)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lon'] = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'Vicinity'] = results[0]['vicinity']
    
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
hotel_df

c:\users\belac\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving Results for Index 437: urumqi.
Closest hotel is 海航产权酒店.
------------
Retrieving Results for Index 571: karkaralinsk.


c:\users\belac\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
c:\users\belac\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


Closest hotel is Hôtel Les Amandiers.
------------
Retrieving Results for Index 613: dandong.
Closest hotel is مزرعه كامل موسى.
------------


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp (f),Wind Speed (mph),Hotel Name,Vicinity
437,urumqi,0,CN,1583761945,83,18.196038,109.490047,78.800,4.47,海航产权酒店,"旅游度假区, Sanya"
571,karkaralinsk,0,MA,1583761956,9,29.717878,-8.974418,73.292,6.22,Hôtel Les Amandiers,Tafraoute
613,dandong,0,SA,1583761960,22,28.378678,41.289641,70.412,6.11,مزرعه كامل موسى,الرديفة


In [6]:
print(type(hotel_df))

<class 'pandas.core.frame.DataFrame'>


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

# Assign the marker layer to a variable

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df5.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map